In [458]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize, least_squares
from sklearn.linear_model import LinearRegression, Lasso
from python_module.pricing_model import SABRModel, BlackScholesModel

In [459]:
# -------
# PARAMETERS
# -------
F = 100
T = 1
alpha = 0.1
beta = 1
rho = -0.
nu = 0.0001
n_steps = 250
n_paths = 100

In [460]:
# -------
# MONTECARLO
# -------
spot, vol = SABRModel.compute_montecarlo(F=F, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, n_steps=n_steps, n_paths=n_paths, seed=False)
spot = np.round(spot, 2)

In [461]:
# -------
# PDF FROM TERMINAL DISTRIBUTION WITH TRUE PARAMETERS
# -------
dk = 1
r  = 0
pdf_dict = dict()
for K in spot.iloc[-1].sort_values():
    a = SABRModel.compute_option(F, K+dk, T, alpha, beta, rho, nu, r, option_type='call')['price']
    b = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['price']
    c = SABRModel.compute_option(F, K-dk, T, alpha, beta, rho, nu, r, option_type='call')['price']
    pdf_dict[K] = (a -2*b + c)/(dk**2)

In [462]:
# -------
# PDF FROM TERMINAL DISTRIBUTION WITH WRONG PARAMETERS
# -------

# Parameter overriding 
alpha = 0.1
beta = 1
rho = 0.
nu = 1


dk = 1
r  = 0
pdf_dict2 = dict()
for K in spot.iloc[-1].sort_values():
    a = SABRModel.compute_option(F, K+dk, T, alpha, beta, rho, nu, r, option_type='call')['price']
    b = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['price']
    c = SABRModel.compute_option(F, K-dk, T, alpha, beta, rho, nu, r, option_type='call')['price']
    pdf_dict2[K] = (a -2*b + c)/(dk**2)

In [463]:
# -------
# COMPARING LOG LIKELYHOOD COMPUTED FROM PDF WITH TRUE AND WRONG PARAMETERS
# -------
pdf_true_param = pd.Series(pdf_dict, name='pdf_true_param')
pdf_wrong_param = pd.Series(pdf_dict2, name='pdf_wrong_param')
df = pd.concat([pdf_true_param, pdf_wrong_param], axis=1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['pdf_true_param'].values, mode='lines+markers', name='pdf_true_param'))
fig.add_trace(go.Scatter(x=df.index, y=df['pdf_wrong_param'].values, mode='lines+markers', name='pdf_wrong_param'))
fig.show()

df = np.log(df)
mle_would_work = df.sum()['pdf_true_param'] >= df.sum()['pdf_wrong_param']

print(f'sum of log likelyhood with true parameters > sum of log likelyhood with wrong parameters : {mle_would_work}')

sum of log likelyhood with true parameters > sum of log likelyhood with wrong parameters : True
